```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. see different plots: https://scikit-plot.readthedocs.io/en/stable/metrics.html
2. https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0118432
3. https://dl.acm.org/doi/10.1145/1143844.1143874
4. https://dl.acm.org/doi/10.1145/65943.65945
5. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4349800/
6. (free): http://www.blogspot.udec.ugto.saedsayad.com/docs/ROC101.pdf
7. https://developers.google.com/machine-learning/crash-course/classification/thresholding
```

# Classification Metrics and Evaluation

## Table of Contents

- [Confusion Matrix](#matrix)
- [Classification Metrics](#metrics) - Accuracy, Precision, Recall, F-Score...
- [Classification Report](#report)
- [Diagnostic tools for probabilistic forecast in binary classification](#prob)
    - [ROC Curve and AUC (best for balanced classes)](#roc)
    - [Precision-Recall Curve (best for imbalanced classes)](#prc)
    - [PROBLEM: Compare ROC Curve and PR Curve for Imbalanced Classes](#imbalanced)
    - [Cumulative Gains Plot](#gain)
    - [Lift Curve](#lift)
- [Class Balance](#class)
- [Class Prediction Error](#class_err)
- [Discrimination Threshold](#threshold)
    - [Threshold-Moving for Imbalanced Classification](#threshold_imb)
    - [Discrimination Thershold Plot](#threshold_plot)
    - [Optimal Threshold for ROC Curve](#threshold_roc)
    - [Optimal Threshold for Precision-Recall Curve](#threshold_prc)
    - [Optimal Threshold Tuning](#threshold_tune)

In [ ]:
# Connect with underlying Python code
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../src')

In [ ]:
from datasets import (
    get_dataset
)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# To eliminate Yellowbrick warming: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
import matplotlib.font_manager

import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
df = get_dataset('pima-indians-diabetes')
df.head()

In [ ]:
# Missing values?
df.isnull().mean().value_counts()

In [ ]:
y = df[['class']]
X = df.drop(columns=['class'])

In [ ]:
# Are the classes balanced
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42, n_estimators=1000, n_jobs=-1, max_depth=None)
rfc.fit(X_train, y_train.values.ravel())

In [ ]:
y_predict = rfc.predict(X_test)
y_predict.shape

In [ ]:
y_predict = y_predict.reshape(y_predict.shape[0], 1)
y_predict.shape

---
<a id='matrix'></a>

## Confusion Matrix
A **confusion matrix** can aid in understanding how a classifier performs.

A **binary classifier** can have four classification results: `true positives (TP)`, `true negatives (TN)`, `false positives (FP)`, and `false negatives (FN)`. The first two are correct classifications.

Here is a common example for remembering the other results. Assuming positive means pregnant and negative is not pregnant, a `false positive` is like claiming a man is pregnant. A `false negative` is claiming that a pregnant woman is not (when she is clearly showing). These last two types of errors are referred to as `type 1` and `type 2` errors, respectively.

<img src="images/type1_type2_errors.png" alt="" style="width: 650px;"/>

Another way to remember these is that `P (for false positive)` has one straight line in it (`type 1 error`), and `N (for false negative)` has two vertical lines in it (`type 2 error`).

In [ ]:
tp = ((y_test == 1) & (y_test == y_predict)).sum().values[0]
tn = ((y_test == 0) & (y_test == y_predict)).sum().values[0]
fp = ((y_test == 0) & (y_test != y_predict)).sum().values[0]
fn = ((y_test == 1) & (y_test != y_predict)).sum().values[0]

In [ ]:
f'tp = {tp}, tn = {tn}, fp = {fp}, fn = {fn}'

Well-behaving classifiers ideally have high counts in the true diagonal. We can create a DataFrame using the sklearn `confusion_matrix` function.

In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=[
        "Predicted 0",
        "Predicted 1",
    ],
    index=["True 0 (Healthy)", "True 1 (Diabetes)"],
)

In [ ]:
from yellowbrick.classifier import (
    ConfusionMatrix,
)

mapping = {0: "Healthy", 1: "Diabetes"}

fig, ax = plt.subplots(figsize=(6, 6))
cm_viz = ConfusionMatrix(
    rfc,
    classes=["Healthy", "Diabetes"],
    label_encoder=mapping,
)

cm_viz.score(X_test, y_test)
cm_viz.poof()
fig.savefig("images/confusion_matrx.png", dpi=300)

**Confusion matrix:** The upper left and lower right are correct classifications. The lower left is false negative. The upper right is false positive.

In [ ]:
def tn(y_test, y_pred): return confusion_matrix(y_test, y_pred)[0, 0]
def fp(y_test, y_pred): return confusion_matrix(y_test, y_pred)[0, 1]
def fn(y_test, y_pred): return confusion_matrix(y_test, y_pred)[1, 0]
def tp(y_test, y_pred): return confusion_matrix(y_test, y_pred)[1, 1]
def PPV(y_test, y_pred): return tp(y_test, y_pred) / (tp(y_test, y_pred) + fp(y_test, y_pred))
def TPR(y_test, y_pred): return tp(y_test, y_pred) / (tp(y_test, y_pred) + fn(y_test, y_pred))
def TNR(y_test, y_pred): return tn(y_test, y_pred) / (tn(y_test, y_pred) + fp(y_test, y_pred))
def NPV(y_test, y_pred): return tn(y_test, y_pred) / (tn(y_test, y_pred) + fn(y_test, y_pred))
def BA(y_test, y_pred): return (TPR(y_test, y_pred) + TNR(y_test, y_pred)) / 2
def F1(y_test, y_pred): return 2 * PPV(y_test, y_pred) * TPR(y_test, y_pred) / (PPV(y_test, y_pred) + TPR(y_test, y_pred))
def F2(y_test, y_pred):
    beta_sqared = 2**2
    return (1 + beta_sqared) * PPV(y_test, y_pred) * TPR(y_test, y_pred) / ((beta_sqared * PPV(y_test, y_pred)) + TPR(y_test, y_pred))

In [ ]:
# Confusion matrix
print(tp(y_test, y_pred), fp(y_test, y_pred))
print(fn(y_test, y_pred), tn(y_test, y_pred))

---
<a id='metrics'></a>

## Metrics
The [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) module includes metrics to evaluate classification models. **Metric functions ending in `loss` or `error` should be minimized. Functions ending in `score` should be maximized.**

In [ ]:
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

The `sklearn.metrics` module implements many **common classification metrics**, including:
- **accuracy:** Percent of correct predictions
- **average_precision:** Precision recall curve summary
- **f1:** Harmonic mean of precision and recall
- **neg_log_loss:** Logistic or cross-entropy loss (model must support `predict_proba`)
- **precision:** Ability to find only relevant samples (not label a negative as a positive)
- **recall:** Ability to find all positive samples
- **roc_auc:** Area under the receiver operator characteristic curve

f1, precision, and recall all support the following suffixes for `multiclass classifers`:
- **_micro:** Global weighted average of metric
- **_macro:** Unweighted average of metric
- **_weighted:** Multiclass weighted average of metric
- **_samples:** Per sample metric

These strings can be used as the `scoring` parameter when doing grid search, or you can use functions from the `sklearn.metrics` module that have the same names as the strings but end in `_score`. See the following note for examples.

### Accuracy
**Accuracy** is the percentage of correct classifications. What is good accuracy? It depends. If I’m predicting fraud (which usually is a rare event, say 1 in 10,000), I can get very high accuracy by always predicting not fraud. But this model is not very useful. Looking at other metrics and the cost of predicting a false positive and a false negative can help us determine if a model is decent. See also `balanced accuracy` (below) for imbalanced classes.

In [ ]:
(tp + tn) / (tp + tn + fp + fn)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

### F-score vs Accuracy
There are a number of metrics which can be used to evaluate a binary classification model, and accuracy is one of the simplest to understand. `Accuracy` is defined as simply the number of correctly categorized examples divided by the total number of examples. `Accuracy` can be useful but does not take into account the subtleties of class imbalances, or differing costs of false negatives and false positives.

The `F1-score` is useful:

- where there are either differing costs of false positives or false negatives
- or where there is a large class imbalance.

The `accuracy` has the advantage that it is very easily interpretable, but the disadvantage that it is not robust when the data is unevenly distributed, or where there is a higher cost associated with a particular type of error.

See also `balanced accuracy` below.

### Recall
**Recall** (also called `sensitivity` or `True Positive Rate`) is the percentage of positive values correctly classified. It is ability to find all positive samples. **Find everything you need**. `How many relevant items are selected?`

It describes how good the model is at predicting the positive class when the actual outcome is positive.

In [ ]:
# Recall, Hit rate, Sensitivity, True Positive Rate
TPR = tp / (tp + fn)
TPR

In [ ]:
from sklearn.metrics import recall_score

recall_score(y_test, y_predict)

<img src="images/precision_recall.png" alt="" />

- **precision:** Ability to find only relevant samples (not label a negative as a positive)
- **recall:** Ability to find all positive samples

From: https://en.wikipedia.org/wiki/F1_score

### Precision
**Precision** is the percent of positive predictions that were correct. It is ability to find only relevant samples (not label a negative as a positive). **Limit junk level**. `How many selected items are relevant?`

It describes how good a model is at predicting the positive class.

In [ ]:
# Precision, Positive Predictive Value
PPV = tp / (tp + fp)
PPV

In [ ]:
from sklearn.metrics import precision_score

precision_score(y_test, y_predict)

### Specificity

<img src="images/sensitivity_and_specificity.png" alt="" />

From: https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [ ]:
# Selectivity, Specificity, True Negative Rate
TNR = tn / (tn + fp)
TNR

### False Positive Rate
In statistics, when performing multiple comparisons, a `false positive ratio` (also known as `fall-out` or `false alarm ratio`) is the probability of falsely rejecting the null hypothesis for a particular test. 

It summarizes how often a positive class is predicted when the actual outcome is negative. 

From: https://en.wikipedia.org/wiki/False_positive_rate

In [ ]:
# False Positive Rate
FPR = fp / (tn + fp)
FPR

### Negative Predictive Value

From: https://en.wikipedia.org/wiki/Positive_and_negative_predictive_values

In [ ]:
# Negative Predictive Value
NPV = tn / (tn + fn)
NPV

### Balanced Accuracy
From: https://en.wikipedia.org/wiki/Precision_and_recall

In [ ]:
# Balanced Accuracy
BA = (TPR + TNR) / 2
BA

### F-Score, F1-Score
The **F-score** is a way of combining the `precision` and `recal`l of the model, and it is defined as the `harmonic mean` of the model’s precision and recall.

It is possible to adjust the F-score to give more importance to precision over recall, or vice-versa. Common adjusted F-scores are the F0.5-score and the F2-score, as well as the standard F1-score.

More and examples: https://deepai.org/machine-learning-glossary-and-terms/f-score

In [ ]:
pre = tp / (tp + fp)
rec = tp / (tp + fn)
(2 * pre * rec) / (pre + rec)

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)

### F beta Score / Metric
Two other commonly used **F measures** are the **F2 measure**, which weights recall higher than precision, and the **F0.5 measure**, which puts more emphasis on precision than recall.

From: https://en.wikipedia.org/wiki/Precision_and_recall

In [ ]:
# F2 Score -> More on Recall
beta_sqared = 2**2
F2 = (1 + beta_sqared) * PPV * TPR / ((beta_sqared * PPV) + TPR) 
F2

In [ ]:
# F0.5 Score -> More on Precision
beta_sqared = 0.5**2
F05 = (1 + beta_sqared) * PPV * TPR / ((beta_sqared * PPV) + TPR) 
F05

---
<a id='report'></a>

## Classification Report
Yellowbrick has a **classification report** showing `precision`, `recall`, and `f1` scores for both positive and negative values.

In [ ]:
from yellowbrick.classifier import (
    ClassificationReport,
)

fig, ax = plt.subplots(figsize=(12, 6))
cm_viz = ClassificationReport(
    rfc,
    classes=["Healthy", "Diabetes"],
    label_encoder=mapping,
)
cm_viz.score(X_test, y_test)
cm_viz.poof()
fig.savefig("images/classification_report.png", dpi=300)

---
<a id='prob'></a>

## Diagnostic tools for probabilistic forecast in binary classification 

In a classification problem, we may decide to `predict the class values directly`. Alternately, it can be more flexible to `predict the probabilities for each class` instead. The reason for this is to provide the capability to choose and even calibrate the threshold for how to interpret the predicted probabilities.

For example, a default might be to use a threshold of 0.5, meaning that a probability in [0.0, 0.49] is a negative outcome (0) and a probability in [0.5, 1.0] is a positive outcome (1). This threshold can be adjusted to tune the behavior of the model for a specific problem. An example would be `to reduce more of one or another type of error`.

When making a prediction for a binary or two-class classification problem, there are two types of errors that we could make.

- **Type 1 Error: False Positive.** Predict an event when there was no event.
- **Type 2 Error: False Negative.** Predict no event when in fact there was an event.

`By predicting probabilities and calibrating a threshold, a balance of these two concerns can be chosen` by the operator of the model.

For example, in a smog prediction system, we may be far more concerned with having low false negatives than low false positives. A false negative would mean not warning about a smog day when in fact it is a high smog day, leading to health issues in the public that are unable to take precautions. A false positive means the public would take precautionary measures when they didn’t need to.

Two diagnostic tools that help in the interpretation of probabilistic forecast for binary (two-class) classification predictive modeling problems are **ROC Curves** and **Precision-Recall Curves**.

- **ROC Curves** summarize the trade-off between the true positive rate and false positive rate for a predictive model using different probability thresholds. ROC curves should be used when there are roughly equal numbers of observations for each class. The reason for this recommendation is that ROC curves present an optimistic picture of the model on datasets with a class imbalance. Some go further and suggest that using a ROC curve with an imbalanced dataset might be deceptive and lead to incorrect interpretations of the model skill. The main reason for this optimistic picture is because of the use of true negatives in the False Positive Rate in the ROC Curve and the careful avoidance of this rate in the Precision-Recall curve.
- **Precision-Recall Curves** summarize the trade-off between the true positive rate and the positive predictive value for a predictive model using different probability thresholds. Precision-Recall curves should be used when there is a moderate to large class imbalance.

`ROC Curves are appropriate when the observations are balanced between each class, whereas Precision-Recall Curves are appropriate for imbalanced datasets`.

---
<a id='roc'></a>

## ROC Curve and AUC (best for balanced classes)
A **[Receiver Operating Characteristic (ROC) Curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)** illustrates how the classifier performs by tracking the true positive rate (`TPR`, recall, sensitivity) as the false positive rate (`FPR`, inverted specificity) changes. It is a plot of the false positive rate (x-axis) versus the true positive rate (y-axis) for a number of different candidate threshold values between 0.0 and 1.0. Put another way, it plots the false alarm rate versus the hit rate.

A rule of thumb is that the plot should bulge out toward the top-left corner. A plot that is to the left and above another plot indicates better performance. The diagonal in this plot indicates the behavior of a random guessing classifier. By taking the **Area Under the Curve (AUC)**, you get a metric for evaluating the performance.

The ROC curve is a useful tool for a few reasons:

- The curves of different models can be compared directly in general or for different thresholds.
- The **Area Under the Curve (AUC)** can be used as a summary of the model skill.
- The shape of the curve contains a lot of information, including what we might care about most for a problem, the expected false positive rate, and the false negative rate.

To make this clear:

- Smaller values on the x-axis of the plot indicate lower false positives and higher true negatives.
- Larger values on the y-axis of the plot indicate higher true positives and lower false negatives.

If you are confused, remember, when we predict a binary outcome, it is either a correct prediction (true positive) or not (false positive). There is a tension between these options, the same with true negative and false negative.

A `skilful model` will assign a higher probability to a randomly chosen real positive occurrence than a negative occurrence on average. This is what we mean when we say that the model has skill. Generally, `skilful models are represented by curves that bow up to the top left of the plot`.

A `no-skill classifier` is one that cannot discriminate between the classes and would predict a random class or a constant class in all cases. A model with no skill is represented at the point (0.5, 0.5). `A model with no skill at each threshold is represented by a diagonal line from the bottom left of the plot to the top right and has an AUC of 0.5`.

`A model with perfect skill is represented at a point (0,1)`. A model with perfect skill is represented by a line that travels from the bottom left of the plot to the top left and then across the top to the top right.

An operator may plot the ROC curve for the final model and choose a threshold that gives a desirable balance between the false positives and false negatives.

**Below** there is an exacmple on how ROC Curve can be deceptive when using for imballanced classification problem.

In [ ]:
y_test.head()

In [ ]:
y_test.values.ravel()

In [ ]:
y_predict[:5]

We can plot a **ROC Curve** for a model in Python using the [roc_curve()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) scikit-learn function. The function takes both the true outcomes (0,1) from the test set and the predicted probabilities for the 1 class. The function returns the false positive rates for each threshold, true positive rates for each threshold and thresholds.

In [ ]:
# Calculate probabilities for both classes
y_predict_proba = rfc.predict_proba(X_test)
y_predict_proba.shape

In [ ]:
y_predict_proba[:5]

In [ ]:
# Keep probabilities for the positive outcome only (Class 1)
y_predict_proba_class_1 = y_predict_proba[:, 1]
y_predict_proba_class_1[:5]

In [ ]:
from sklearn.metrics import roc_curve

# Calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test.values.ravel(), y_predict_proba_class_1)

In [ ]:
fpr[:5]

In [ ]:
tpr[:5]

In [ ]:
thresholds

The **AUC for the ROC** can be calculated using the [roc_auc_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) function. Like the `roc_curve()` function, the AUC function takes both the true outcomes (0,1) from the test set and the predicted probabilities for the 1 class. It returns the AUC score between 0.0 and 1.0 for no skill and perfect skill respectively.

In [ ]:
# Calculate AUC
auc = roc_auc_score(y_test.values.ravel(), y_predict_proba_class_1)
print('AUC: %.3f' % auc)

In [ ]:
# generate a no skill prediction data (majority class) data (for the plot)
# always predicts 0
no_skill_proba_class_1 = [0 for _ in range(len(y_test.values.ravel()))]
no_skill_auc = roc_auc_score(y_test.values.ravel(), no_skill_proba_class_1)
print('AUC: %.3f' % no_skill_auc)

# Calculate roc curve
no_skill_fpr, no_skill_tpr, no_skill_thresholds = roc_curve(y_test.values.ravel(), no_skill_proba_class_1)

In [ ]:
no_skill_fpr[:5], no_skill_tpr[:5]

In [ ]:
# plot the roc curve for the model
pyplot.plot(no_skill_fpr, no_skill_tpr, linestyle='--', label='No Skill')
pyplot.plot(fpr, tpr, marker='.', label='RandomForestClassifier')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')

# show the legend
pyplot.legend()

# show the plot
pyplot.show()

Given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision if not using `random_state`, the results may vary. **Consider running the example a few times and compare the average outcome**.

#### ROC Curve Example
A complete example of calculating the ROC curve and ROC AUC for a Logistic Regression model on a small test problem is listed below.

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)

# generate a no skill prediction (majority class)
# always predicts 0
ns_probs = [0 for _ in range(len(testy))]

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
lr_probs = model.predict_proba(testX)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)

# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)

# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')

# show the legend
pyplot.legend()

# show the plot
pyplot.show()

### ROCAUC with Yellowbrick

In [ ]:
from yellowbrick.classifier import ROCAUC

fig, ax = plt.subplots(figsize=(12, 12))
roc_viz = ROCAUC(rfc)
roc_viz.score(X_test, y_test)

roc_viz.poof()
fig.savefig("images/roc.png", dpi=300)

---
<a id='prc'></a>

## Precision-Recall Curve (best for imbalanced classes)
`The ROC curve may be overly optimistic for imbalanced classes`. Another option for evaluating classifiers is using a **Precision-Recall (PR) Curve**. Classification is a balancing act of finding everything you need (`recall`) while limiting the junk results (`precision`). This is typically a trade-off. As recall goes up, precision usually goes down and vice versa.

Reviewing both precision and recall is useful in cases where there is an imbalance in the observations between the two classes. Specifically, there are many examples of no event (class 0) and only a few examples of an event (class 1). The reason for this is that typically **the large number of class 0 examples means we are less interested in the skill of the model at predicting class 0 correctly, e.g. high true negatives**.

**Key to the calculation of `precision` and `recall` is that the calculations do not make use of the true negatives. It is only concerned with the correct prediction of the minority class, class 1**.

A `precision-recall curve` is a plot of the precision (y-axis) and the recall (x-axis) for different thresholds, much like the `ROC curve`.

A no-skill classifier is one that cannot discriminate between the classes and would predict a random class or a constant class in all cases. The no-skill line changes based on the distribution of the positive to negative classes. It is a `horizontal line` with the value of the ratio of positive cases in the dataset. For a balanced dataset, this is 0.5.

`A model with perfect skill is depicted as a point at (1,1)`. A skilful model is represented by a curve that bows towards (1,1) above the flat line of no skill.

There are also composite scores that attempt to summarize the precision and recall; two examples include:

- **F-Measure or F1 score:** that calculates the harmonic mean of the precision and recall (harmonic mean because the precision and recall are rates).
- **Area Under Curve:** like the AUC, summarizes the integral or an approximation of the area under the precision-recall curve.

In terms of model selection, F-Measure summarizes model skill for a specific probability threshold (e.g. 0.5), whereas the area under curve summarize the skill of a model across thresholds, like ROC AUC.

This makes precision-recall and a plot of precision vs. recall and summary measures **useful tools for binary classification problems that have an imbalance in the observations for each class**.

Precision and recall can be calculated in `scikit-learn`. The precision and recall can be calculated for thresholds using the [precision_recall_curve()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html) function that takes the true output values and the probabilities for the positive class as output and returns the precision, recall and threshold values. The F-Measure can be calculated by calling the [f1_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) function that takes the true class values and the predicted class values as arguments.

In [ ]:
from sklearn.metrics import precision_recall_curve

# Calculate roc curve
precision, recall, thresholds = precision_recall_curve(y_test.values.ravel(), y_predict_proba_class_1)

In [ ]:
precision[:5], recall[:5], thresholds[:5]

In [ ]:
len(precision), len(recall), len(thresholds)

In [ ]:
from sklearn.metrics import f1_score

# Calculate F1 score
f1_score = f1_score(testy, yhat)

The area under the precision-recall curve can be approximated by calling the [auc()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html) function and passing it the recall (x) and precision (y) values calculated for each threshold.

In [ ]:
from sklearn.metrics import auc

# Calculate precision-recall AUC
auc_score = auc(recall, precision)
auc_score

When plotting precision and recall for each threshold as a curve, it is important that `recall is provided as the x-axi`s and `precision is provided as the y-axis`.

In [ ]:
# Summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (f1_score, auc_score))

# Plot the precision-recall curves
no_skill = len(y_test.values.ravel()[y_test.values.ravel()==1]) / len(y_test.values.ravel())
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', label='Logistic')

# Axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')

# Show the legend
pyplot.legend()

# Show the plot
pyplot.show()

Given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision if not using `random_state`, the results may vary. **Consider running the example a few times and compare the average outcome**.

#### Precision-Recall Curve Example
The complete example of calculating precision-recall curves for a Logistic Regression model is listed below. The precision-recall curve plot is then created showing the precision/recall for each threshold for a logistic regression model (orange) compared to a no skill model (blue).

In [ ]:
# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot

# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
lr_probs = model.predict_proba(testX)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

# predict class values
yhat = model.predict(testX)
lr_precision, lr_recall, _ = precision_recall_curve(testy, lr_probs)
lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)

# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

# plot the precision-recall curves
no_skill = len(testy[testy==1]) / len(testy)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')

# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')

# show the legend
pyplot.legend()

# show the plot
pyplot.show()

### PrecisionRecallCurve with Yellowbrick

In [ ]:
from sklearn.metrics import (
    average_precision_score,
)

average_precision_score(y_test, y_predict)

In [ ]:
from yellowbrick.classifier import (
    PrecisionRecallCurve,
)

fig, ax = plt.subplots(figsize=(12, 8))
viz = PrecisionRecallCurve(
    RandomForestClassifier(max_depth=3)
)
viz.fit(X_train, y_train)
print(viz.score(X_test, y_test))
viz.poof()
fig.savefig("images/precision_recall_curve.png", dpi=300)

---
<a id='imbalanced'></a>

### PROBLEM: Compare ROC Curve and PR Curve for Imbalanced Classes
Below is the same **ROC Curve** example with a modified problem where there is a ratio of about 100:1 ratio of class=0 to class=1 observations (specifically Class0=985, Class1=15).

In [ ]:
# roc curve and auc on an imbalanced dataset
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99,0.01], random_state=1)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)

# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(testy))]

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
lr_probs = model.predict_proba(testX)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)

# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)

# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')

# show the legend
pyplot.legend()

# show the plot
pyplot.show()

Running the example suggests that the model has skill. Indeed, it has skill, but `all of that skill is measured as making correct true negative predictions and there are a lot of negative predictions to make`. If you review the predictions, you will see that the model predicts the majority class (class 0) in all cases on the test set. **The score is very misleading**.

We can also repeat the test of the same model on the same dataset and calculate a **precision-recall curve** and statistics instead. The complete example is listed below.

In [ ]:
# precision-recall curve and f1 for an imbalanced dataset
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot

# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99,0.01], random_state=1)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
lr_probs = model.predict_proba(testX)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

# predict class values
yhat = model.predict(testX)

# calculate precision and recall for each threshold
lr_precision, lr_recall, _ = precision_recall_curve(testy, lr_probs)

# calculate scores
lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)

# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

# plot the precision-recall curves
no_skill = len(testy[testy==1]) / len(testy)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')

# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')

# show the legend
pyplot.legend()

# show the plot
pyplot.show()

We can see that the model is penalized for predicting the majority class in all cases. The scores show that the model that looked good according to the ROC Curve is in fact barely skillful when considered using precision and recall that focus on the positive class.

The plot of the precision-recall curve highlights that the model is just barely above the no skill line for most thresholds. This is possible because the model predicts probabilities and is uncertain about some cases. These get exposed through the different thresholds evaluated in the construction of the curve, flipping some class 0 to class 1, offering some precision but very low recall.

---
<a id='gain'></a>

## Cumulative Gains Plot
A **cumulative gains plot** can be used `to evaluate a binary classifier`. It models the `true positive rate (sensitivity)` against the `support rate (fraction of positive predictions)`. The intuition behind this plot is to sort all classifications by predicted probability. Ideally there would be a clean cut that divides positive from negative samples. If the first 10% of the predictions has 30% of the positive samples, you would plot a point from (0,0) to (.1, .3). You continue this process through all of the samples.

`A common use for this is determining customer response`. The cumulative gains curve plots the support or predicted positive rate along the x-axis. Our chart labels this as `"Percentage of sample"`. It plots the sensitivity or true positive rate along the y-axis. This is labeled as `"Gain"` in our plot.

**If you wanted to contact 90% of the customers that would respond (sensitivity), you can trace from .9 on the y-axis to the right until you hit that curve. The x-axis at that point will indicate how many total customers you need to contact (support) to get to 90%.**

In this case we aren’t contacting customers that would respond to a survey but predicting if someone has diabetes. If we ordered all patients according to our model by how likely they are to have diabetes, if you took the first 60% of them, you would have 90% of the survivors. If you have an associated cost per contact and revenue per response, you can calculate what the best number is.

In general, a model that is to the left and above another model is a better model. The best models are lines that go up to the top (if 10% of the samples are positive, it would hit at (.1, 1)) and then directly to the right. If the plot is below the baseline, we would do better to randomly assign labels that use our model.

In [ ]:
import scikitplot as skplt

fig, ax = plt.subplots(figsize=(12, 12))

y_probas = rfc.predict_proba(X_test)

skplt.metrics.plot_cumulative_gain(
    y_test, 
    y_probas, 
    ax=ax,
)

fig.savefig("images/cumulative_gains_plot.png", dpi=300, bbox_inches="tight",)

---
<a id='lift'></a>

## Lift Curve
A **lift curve** is another way of looking at the information in a cumulative gains plot. The `lift` is how much better we are doing than the baseline model. In our plot below, we can see that if we sorted our Diabetes patients by the has-diabetes probability and took the first 20% of them, our lift would be about 2.2 times (the gain divided by sample percent) better than randomly choosing patients. We would get 2.2 times as many patients.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

skplt.metrics.plot_lift_curve(
    y_test, 
    y_probas, 
    ax=ax,
)

fig.savefig("images/lift_curve.png", dpi=300, bbox_inches="tight",)

---
<a id='class'></a>

## Class Balance
**When the relative class sizes are different, `accuracy` is not a good evaluation metric.** When splitting up the data into training and test sets, use `stratified sampling` so the sets keep a relative proportion of the classes. The `test_train_split` function does this when you set the `stratify` parameter to the labels.

In [ ]:
from yellowbrick.classifier import ClassBalance

fig, ax = plt.subplots(figsize=(6, 6))
cb_viz = ClassBalance(labels=["Healthy", "Diabetes"])
cb_viz.fit(y_test.values.reshape(-1, 1).flatten())
cb_viz.poof()
fig.savefig("images/class_balance.png", dpi=300)

In [ ]:
y_test

In [ ]:
y_test.values.reshape(-1, 1).flatten()

---
<a id='class_err'></a>

## Class Prediction Error
The **class prediction error plot** from Yellowbrick is a bar chart that visualizes a `confusion matrix`.

In [ ]:
y_test_arr = y_test.values.reshape(-1, 1).flatten()
y_test_arr

In [ ]:
from yellowbrick.classifier import (
    ClassPredictionError,
)

fig, ax = plt.subplots(figsize=(12, 8))
cpe_viz = ClassPredictionError(
    rfc, classes=["Healthy", "Diabetes"]
)
cpe_viz.score(X_test, y_test_arr)
cpe_viz.poof()
fig.savefig("images/class_prediction_error.png", dpi=300)

**Class prediction error:** At the top of the left bar are people who are healthy, but we predicted that they have diabetes (`false positive`). At the bottom of the right bar are people who has diabetes, but the model predicted they are healthy (`false negative`).

---
<a id='threshold'></a>

## Discrimination Threshold
Many machine learning algorithms are capable of predicting a probability or scoring of class membership, and this must be interpreted before it can be mapped to a crisp class label. This is useful generally as it provides a measure of the certainty or uncertainty of a prediction. It also provides additional granularity over just predicting the class label that can be interpreted.

The decision for converting a predicted probability or scoring into a class label is governed by a parameter referred to as the **decision threshold**, **discrimination threshold**, or simply the **threshold**. The default value for the threshold is 0.5 for normalized predicted probabilities or scores in the range between 0 or 1.

For example, on a binary classification problem with class labels 0 and 1, normalized predicted probabilities and a threshold of 0.5, then values less than the threshold of 0.5 are assigned to class 0 and values greater than or equal to 0.5 are assigned to class 1:

- Prediction < 0.5 = Class 0
- Prediction >= 0.5 = Class 1

The problem is that the default threshold may not represent an optimal interpretation of the predicted probabilities. This might be the case for a number of reasons, such as:

- The predicted probabilities are not calibrated, e.g. those predicted by an SVM or decision tree.
- The metric used to train the model is different from the metric used to evaluate a final model.
- The class distribution is severely skewed (imbalanced classification problem).
- The cost of one type of misclassification is more important than another type of misclassification.

Worse still, some or all of these reasons may occur at the same time. In such cases, the default threshold can result in poor performance. As such, **a simple and straightforward approach to improving the performance of a classifier that predicts probabilities on an imbalanced classification problem is to tune the threshold** used to map probabilities to class labels.

In some cases, such as when using ROC Curves and Precision-Recall Curves, the best or optimal threshold for the classifier can be calculated directly. In other cases, it is possible to use a grid search to tune the threshold and locate the optimal value.

---
<a id='threshold_imb'></a>

### Threshold-Moving for Imbalanced Classification

There are many techniques that may be used to address an imbalanced classification problem, such as resampling the training dataset and developing customized version of machine learning algorithms.

Nevertheless, perhaps the simplest approach to handle a severe class imbalance is to change the decision threshold. Although simple and very effective, this technique is often overlooked by practitioners and research academics alike.

You may use: 
- **ROC Curves** to analyze the predicted probabilities of a model and **ROC AUC** scores to compare and select a model, although you require crisp class labels from your model. How do you choose the threshold on the ROC Curve that results in the best balance between the true positive rate and the false positive rate?
- **Precision-Recall Curves** to analyze the predicted probabilities of a model, **Precision-Recall AUC** to compare and select models, and require crisp class labels as predictions. How do you choose the threshold on the Precision-Recall Curve that results in the best balance between precision and recall?
- a probability-based metric to train, evaluate, and compare models like **log loss (cross-entropy)** but require crisp class labels to be predicted. How do you choose the optimal threshold from predicted probabilities more generally?
- have different costs associated with false positive and false negative misclassification, a so-called **cost matrix**, but wish to use and evaluate cost-insensitive models and later evaluate their predictions use a cost-sensitive measure. How do you choose a threshold that finds the best trade-off for predictions using the cost matrix?

The answer to these questions is to search a range of threshold values in order to find the best threshold. In some cases, the optimal threshold can be calculated directly. Tuning or shifting the decision threshold in order to accommodate the broader requirements of the classification problem is generally referred to as **threshold-moving**, **threshold-tuning**, or simply **thresholding**.

**Trying other methods, such as `sampling`, without trying by simply setting the threshold may be misleading**. `The threshold-moving method uses the original training set to train a model and then moves the decision threshold such that the minority class examples are easier to be predicted correctly.`

trying other methods, such as sampling, without trying by simply setting the threshold may be misleading. The threshold-moving method uses the original training set to train a model] and then moves the decision threshold such that the minority class examples are easier to be predicted correctly.

The process involves first fitting the model on a training dataset and making predictions on a test dataset. The predictions are in the form of normalized probabilities or scores that are transformed into normalized probabilities. Different threshold values are then tried and the resulting crisp labels are evaluated using a chosen evaluation metric. The threshold that achieves the best evaluation metric is then adopted for the model when making predictions on new data in the future.

We can summarize this procedure below.

1. Fit Model on the Training Dataset.
2. Predict Probabilities on the Test Dataset.
3. For each threshold in Thresholds:
    1. Convert probabilities to Class Labels using the threshold.
    1. Evaluate Class Labels.
    1. If Score is Better than Best Score.
        1. Adopt Threshold.
4. Use Adopted Threshold When Making Class Predictions on New Data.

Although simple, `there are a few different approaches to implementing threshold-moving` depending on your circumstance. We will take a look at some of the most common examples in the following sections.

---
<a id='threshold_roc'></a>

### Optimal Threshold for ROC Curve

A **ROC curve** is a diagnostic plot that evaluates a set of probability predictions made by a model on a test dataset. A set of different thresholds are used to interpret the true positive rate and the false positive rate of the predictions on the positive (minority) class, and the scores are plotted in a line of increasing thresholds to create a curve.

The curve is useful to understand the trade-off in the true-positive rate and false-positive rate for different thresholds. The area under the ROC Curve, so-called ROC AUC, provides a single number to summarize the performance of a model in terms of its ROC Curve with a value between 0.5 (no-skill) and 1.0 (perfect skill).

If crisp class labels are required from a model under such an analysis, then **an optimal threshold** is required. This would be `a threshold on the curve that is closest to the top-left of the plot`.

#### Geometric Mean
There are many ways we could locate the threshold with the optimal balance between false positive and true positive rates. The **Geometric Mean** or **G-Mean** is a metric for imbalanced classification that, if optimized, will seek a balance between the `sensitivity/recall/TPR` and the `specificity`.

```
Sensitivity = TruePositive / (TruePositive + FalseNegative)
Sensitivity = True Positive Rate

Specificity = TrueNegative / (FalsePositive + TrueNegative)
Specificity = 1 – False Positive Rate

G-Mean = sqrt(Sensitivity * Specificity)
```
One approach would be to test the model with each threshold returned from the call [roc_auc_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) and select the threshold with the largest G-Mean value.

Given that we have already calculated the Sensitivity (TPR) and the complement to the Specificity when we calculated the ROC Curve, we can calculate the G-Mean for each threshold directly. Once calculated, we can locate the index for the largest G-mean score and use that index to determine which threshold value to use.

The threshold is then used to locate the true and false positive rates, then this point is drawn on the ROC Curve.

If we required crisp class labels from this model, which threshold would achieve the best result? We can see that the point for the optimal threshold is a large black dot and it appears to be closest to the top-left of the plot.

In [ ]:
# ROC Curve for logistic regression model with optimal threshold
from numpy import sqrt
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from matplotlib import pyplot

# generate dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=4)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
yhat = model.predict_proba(testX)

# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

# calculate roc curves
fpr, tpr, thresholds = roc_curve(testy, yhat)

# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))

# locate the index of the largest g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

# plot the roc curve for the model
pyplot.plot([0,1], [0,1], linestyle='--', label='No Skill')
pyplot.plot(fpr, tpr, marker='.', label='Logistic')
pyplot.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()

# show the plot
pyplot.show()

#### Youden's J statistic
It turns out there is a much faster way to get the same result, called the [Youden’s J statistic](https://en.wikipedia.org/wiki/Youden%27s_J_statistic).

The statistic is calculated as:
```
J = Sensitivity + Specificity – 1
```
Given that we have Sensitivity (TPR) and the complement of the specificity (FPR), we can calculate it as:
```
J = Sensitivity + (1 – FalsePositiveRate) – 1
```
Which we can restate as:
```
J = TruePositiveRate – FalsePositiveRate
```
We can then choose the threshold with the largest J statistic value.

In [ ]:
# ROC Curve for logistic regression model with optimal threshold
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

# generate dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=4)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
yhat = model.predict_proba(testX)

# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

# calculate roc curves
fpr, tpr, thresholds = roc_curve(testy, yhat)

# get the best threshold
J = tpr - fpr
ix = argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))

# plot the roc curve for the model
pyplot.plot([0,1], [0,1], linestyle='--', label='No Skill')
pyplot.plot(fpr, tpr, marker='.', label='Logistic')
pyplot.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()

# show the plot
pyplot.show()

This threshold could then be used when making probability predictions in the future that must be converted from probabilities to crisp class labels.
```
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 2)
clf.fit(X_train,y_train)

# default threshold is 0.5
# y_pred = clf.predict(X_test)  

# set threshold as 0.3
y_pred = (clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool) 
```

---
<a id='threshold_prc'></a>

### Optimal Threshold for Precision-Recall Curve
Unlike the ROC Curve, a **Precision-Recall Curve** focuses on the performance of a classifier on the positive (minority class) only.

`Precision` is the ratio of the number of true positives divided by the sum of the true positives and false positives. It describes how good a model is at predicting the positive class. `Recall` is calculated as the ratio of the number of true positives divided by the sum of the true positives and the false negatives. Recall is the same as sensitivity.

A **Precision-Recall Curve** is calculated by creating crisp class labels for probability predictions across a set of thresholds and calculating the precision and recall for each threshold. A line plot is created for the thresholds in ascending order with recall on the x-axis and precision on the y-axis.

A `no-skill model` is represented by a horizontal line with a precision that is the ratio of positive examples in the dataset (e.g. TP / (TP + TN)), or 0.01 on our synthetic dataset. `Perfect skill classifier` has full precision and recall with a dot in the top-right corner.

We can use the same model and dataset from the previous section and evaluate the probability predictions for a logistic regression model using a precision-recall curve. The [precision_recall_curve()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html) function can be used to calculate the curve, returning the precision and recall scores for each threshold as well as the thresholds used.

Running the example below calculates the precision and recall for each threshold and creates a precision-recall plot showing that the model has some skill across a range of thresholds on this dataset. If we required crisp class labels from this model, which threshold would achieve the best result?

If we are interested in a threshold that results in the best balance of precision and recall, then this is the same as optimizing the **F-measure** that summarizes the harmonic mean of both measures.
```
F-Measure = (2 * Precision * Recall) / (Precision + Recall)
```
As in the previous section, the naive approach to finding the optimal threshold would be to calculate the F-measure for each threshold. We can achieve the same effect by converting the precision and recall measures to F-measure directly.
```
# convert to f score
fscore = (2 * precision * recall) / (precision + recall)

# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
```
We can then plot the point on the precision-recall curve.

In [ ]:
# PR Curve for logistic regression model
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot

# generate dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=4)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
yhat = model.predict_proba(testX)

# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

# calculate pr-curve
precision, recall, thresholds = precision_recall_curve(testy, yhat)

# convert to f score
fscore = (2 * precision * recall) / (precision + recall)

# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

# plot the roc curve for the model
no_skill = len(testy[testy==1]) / len(testy)
pyplot.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', label='Logistic')
pyplot.scatter(recall[ix], precision[ix], marker='o', color='black', label='Best')

# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
pyplot.legend()

# show the plot
pyplot.show()

The precision-recall curve is plotted, and this time the threshold with the optimal F-measure is plotted with a larger black dot.

This threshold could then be used when making probability predictions in the future that must be converted from probabilities to crisp class labels.
```
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 2)
clf.fit(X_train,y_train)

# default threshold is 0.5
# y_pred = clf.predict(X_test)  

# set threshold as 0.3
y_pred = (clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool) 
```

---
<a id='threshold_tune'></a>

### Optimal Threshold Tuning
Sometimes, we simply have a model and we wish to know the best threshold directly.

In this case, we can define a set of thresholds and then evaluate predicted probabilities under each in order to find and select the optimal threshold.

First, we can fit a logistic regression model on our synthetic classification problem, then predict class labels and evaluate them using the F-Measure, which is the harmonic mean of precision and recall.

This will use the default threshold of 0.5 when interpreting the probabilities predicted by the logistic regression model.

In [ ]:
# logistic regression for imbalanced classification
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# generate dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=4)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict labels
yhat = model.predict(testX)

# evaluate the model
score = f1_score(testy, yhat)
print('F-Score: %.5f on the test dataset' % score)

Now we can use the same model on the same dataset and instead of predicting class labels directly, we can predict probabilities.

In [ ]:
# predict probabilities
yhat = model.predict_proba(testX)

We only require the probabilities for the positive class.

In [ ]:
yhat[:5]

In [ ]:
# keep probabilities for the positive outcome only
probs = yhat[:, 1]
probs[:5]

Next, we can then define a set of thresholds to evaluate the probabilities. In this case, we will test all thresholds between 0.0 and 1.0 with a step size of 0.001, that is, we will test 0.0, 0.001, 0.002, 0.003, and so on to 0.999.

In [ ]:
from numpy import arange

# define thresholds
thresholds = arange(0, 1, 0.001)
thresholds[:5]

Next, we need a way of using a single threshold to interpret the predicted probabilities.

This can be achieved by mapping all values equal to or greater than the threshold to 1 and all values less than the threshold to 0. We will define a to_labels() function to do this that will take the probabilities and threshold as an argument and return an array of integers in {0, 1}.

In [ ]:
# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

We can then call this function for each threshold and evaluate the resulting labels using the f1_score().

In [ ]:
test_probs = [0.0, 0.1, 0.2, 0.5, 0.7]
test_thresholds = [0.0, 0.2, 0.4, 0.6, 0.8]

In [ ]:
to_labels(np.array(test_probs), test_thresholds[2])

In [ ]:
# evaluate each threshold
scores = [f1_score(testy, to_labels(probs, t)) for t in thresholds]
scores[:5]

We now have an array of scores that evaluate each threshold in our array of thresholds.

All we need to do now is locate the array index that has the largest score (best F-Measure) and we will have the optimal threshold and its evaluation.

In [ ]:
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
# search thresholds for imbalanced classification
from numpy import arange
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

# generate dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=4)

# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)

# predict probabilities
yhat = model.predict_proba(testX)

# keep probabilities for the positive outcome only
probs = yhat[:, 1]

# define thresholds
thresholds = arange(0, 1, 0.001)

# evaluate each threshold
scores = [f1_score(testy, to_labels(probs, t)) for t in thresholds]

# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
# default threshold is 0.5
y_pred = model.predict(testX)
f1_score(testy, y_pred)

In [ ]:
# set threshold as thresholds[ix]
y_pred = (model.predict_proba(testX)[:,1] >= thresholds[ix]).astype(bool)
f1_score(testy, y_pred)

---
<a id='threshold_plot'></a>

### Discrimination Thershold Plot

The **discrimination threshold plot** moves that threshold value between 0 and 100 and shows the impact to `precision`, `recall`, `f1`, and `queue rate`.

This plot can be useful to view the `trade-off between precision and recall`. Assume we are looking for fraud (and considering fraud to be the positive classification). To get high `recall` (catch all of the fraud), we can just classify everything as fraud. But in a bank situation, this would not be profitable and would require an army of workers. To get high `precision` (only catch fraud if it is fraud), we could have a model that only triggers on cases of extreme fraud. But this would miss much of the fraud that might not be as obvious. There is a trade-off here.

The `queue rate` is the percent of predictions above the threshold. You can consider this to be the percent of cases to review if you are dealing with fraud.

If you have the cost for positive, negative, and erroneous calculations, you can determine what threshold you are comfortable with.

The following plot is useful to see what `discrimination threshold` will maximize the `f1` score or adjust `precision` or `recall` to an acceptable number when coupled with the queue rate.

Yellowbrick provides this visualizer. This visualizer shuffles the data and runs 50 trials by default, splitting out 10% for validation.

In [ ]:
from yellowbrick.classifier import (
    DiscriminationThreshold,
)

y_test_arr = y_test.values

fig, ax = plt.subplots(figsize=(12, 12))
dt_viz = DiscriminationThreshold(rfc)
dt_viz.fit(X_test, y_test_arr)
dt_viz.poof()
fig.savefig("images/discrimination_thershold.png", dpi=300)